In [ ]:
import csv
import math

from typing import List

import numpy as np
from matplotlib import pyplot as plt
# import the ZOD DevKit
from zod import ZodFrames
from zod import ZodSequences
from zod import ZodDrives
from datetime import datetime, timezone

# import default constants
import zod.constants as constants
from zod.constants import Camera, Lidar, Anonymization, AnnotationProject

# import useful data classes
from zod.data_classes import LidarData

#for loading zod data
zod_dataset = "/datasets/zod"  # your local path to zod
version = "full"  # "mini" or "full"

# initialize ZodSequences
zod_drives = ZodDrives(dataset_root=zod_dataset, version=version)

import json

zod_frames = ZodFrames(dataset_root=zod_dataset, version=version)
print(len(zod_frames))




Loading infos: 0it [00:00, ?it/s]

In [ ]:
import json
import os
from tqdm import tqdm

parent_folder = "/home/bjangley/VPR/mapillary_utils/zod"  

# Ensure the output folder exists; create it if it doesn't
if not os.path.exists(parent_folder):
    print("Output Folder Does Not Exist")
    exit()  


all_frames_data = {} #dictionary to store all frame data
print("Number of Frames: ",len(zod_frames))

# Iterate through each frame in zod_frames
for i in tqdm(range(len(zod_frames)), desc="Processing frames", unit="frame", total=len(zod_frames)):
    item = zod_frames[i]
    lat = item.metadata.latitude
    long = item.metadata.longitude
    frame_id = item.metadata.frame_id
    country_code = item.metadata.country_code

    # Prepare the data dictionary for this frame
    frame_data = {
        'lat':lat,
        'long':long,
        'country_code': country_code
        }
    
    all_frames_data[frame_id] = frame_data

# Specify the output JSON file path
output_file_path = os.path.join(parent_folder, "frames_v1.json")
# Save the dictionary to a single JSON file
with open(output_file_path, "w") as json_file:
    json.dump(all_frames_data, json_file, indent=4)

print(f"\nAll frames saved to {output_file_path}.")

print("Number of frames processed: ",len(all_frames_data.keys()))
